In [325]:
class Graph:
    def __init__(self,s,d,dis):
        self.s = s
        self.d = d
        self.dis = dis
    

op = open('clustering1.txt','r')

temp = []

for i in op:
    temp.append(i.split(" "))
    
graph = []

nodes = int((temp[0][0]))


for i in temp[1:]:
    graph.append(Graph(int(i[0]), int(i[1]), int(i[2])))
    
print(len(graph))

124750


In [326]:
graph.sort(key = lambda x:x.dis)

In [327]:
#union find

parent = [i for i in range(0,nodes+1)]
rank = [0]*(nodes+1)

def find(x):
    
    if(parent[x] != x):
        parent[x] = find(parent[x])
        
    return parent[x]

def union(x,y):
    
    if(rank[x] > rank[y]):
        parent[y] = parent[x]
        rank[x] +=1
        
    elif(rank[x] < rank[y]):
        parent[x] = parent[y]
        rank[y] += 1
        
    else:
        parent[y] = parent[x]
        rank[x] +=1

# def numClusters():
#     count = 0
#     for i in range(1,nodes+1):
#         if(i == parent[i]):
#             count+=1
#     return count        

In [328]:
i = 0
j = 0

clusters = 4

while j < nodes-clusters:
    
    a = find(graph[i].s)
    b = find(graph[i].d)
    
    if(a != b):
        union(a,b)
        j+=1
    i+=1
    
# print(graph[i].s,graph[i].d,graph[i].dis,numClusters())
    
while(i):
    a = find(graph[i].s)
    b = find(graph[i].d)
    
    if(a != b):
        print(graph[i].s,graph[i].d,graph[i].dis)
        break  
    i+=1

232 403 97 4
414 455 106


# assignment part 2

In [379]:

op1 = open('clustering_big.txt','r')

temp = []

for i in op1:
    temp.append(i.split(" "))
        

bit = int(temp[0][1])
print(bit)

maps = {}

for i in temp[1:]:
    
    intmask = int("".join(i),2)
    
    if( intmask in maps.keys()):
        maps[intmask].append(i)
    else:
        maps[intmask] = []
        maps[intmask].append(i)
        
print(len(maps.keys()))    

24
198788


In [380]:
#storing distances whose less than three bits active
bitmask = [1 << i for i in range(bit)]
bitmask.append(0)

for i in range(bit):
    for j in range(i+1,bit):
        bits = list("0"*bit)
        bits[i] = '1'
        bits[j] = '1'
        bitmask.append(int("".join(bits),2))
        
bitmask.sort()        
print(bitmask)

[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 16, 17, 18, 20, 24, 32, 33, 34, 36, 40, 48, 64, 65, 66, 68, 72, 80, 96, 128, 129, 130, 132, 136, 144, 160, 192, 256, 257, 258, 260, 264, 272, 288, 320, 384, 512, 513, 514, 516, 520, 528, 544, 576, 640, 768, 1024, 1025, 1026, 1028, 1032, 1040, 1056, 1088, 1152, 1280, 1536, 2048, 2049, 2050, 2052, 2056, 2064, 2080, 2112, 2176, 2304, 2560, 3072, 4096, 4097, 4098, 4100, 4104, 4112, 4128, 4160, 4224, 4352, 4608, 5120, 6144, 8192, 8193, 8194, 8196, 8200, 8208, 8224, 8256, 8320, 8448, 8704, 9216, 10240, 12288, 16384, 16385, 16386, 16388, 16392, 16400, 16416, 16448, 16512, 16640, 16896, 17408, 18432, 20480, 24576, 32768, 32769, 32770, 32772, 32776, 32784, 32800, 32832, 32896, 33024, 33280, 33792, 34816, 36864, 40960, 49152, 65536, 65537, 65538, 65540, 65544, 65552, 65568, 65600, 65664, 65792, 66048, 66560, 67584, 69632, 73728, 81920, 98304, 131072, 131073, 131074, 131076, 131080, 131088, 131104, 131136, 131200, 131328, 131584, 132096, 133120, 135168, 139264,

In [381]:
"""
Union-find data structure.
"""

from networkx.utils import groups


class UnionFind:
    """Union-find data structure.

    Each unionFind instance X maintains a family of disjoint sets of
    hashable objects, supporting the following two methods:

    - X[item] returns a name for the set containing the given item.
      Each set is named by an arbitrarily-chosen one of its members; as
      long as the set remains unchanged it will keep the same name. If
      the item is not yet part of a set in X, a new singleton set is
      created for it.

    - X.union(item1, item2, ...) merges the sets containing each item
      into a single larger set.  If any item is not yet part of a set
      in X, it is added to X as one of the members of the merged set.

      Union-find data structure. Based on Josiah Carlson's code,
      https://code.activestate.com/recipes/215912/
      with significant additional changes by D. Eppstein.
      http://www.ics.uci.edu/~eppstein/PADS/UnionFind.py

    """

    def __init__(self, elements=None):
        """Create a new empty union-find structure.

        If *elements* is an iterable, this structure will be initialized
        with the discrete partition on the given set of elements.

        """
        if elements is None:
            elements = ()
        self.parents = {}
        self.weights = {}
        for x in elements:
            self.weights[x] = 1
            self.parents[x] = x

    def __getitem__(self, object):
        """Find and return the name of the set containing the object."""

        # check for previously unknown object
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root

    def __iter__(self):
        """Iterate through all items ever found or unioned by this structure."""
        return iter(self.parents)

    def to_sets(self):
        """Iterates over the sets stored in this structure.

        For example::

            >>> partition = UnionFind("xyz")
            >>> sorted(map(sorted, partition.to_sets()))
            [['x'], ['y'], ['z']]
            >>> partition.union("x", "y")
            >>> sorted(map(sorted, partition.to_sets()))
            [['x', 'y'], ['z']]

        """
        # Ensure fully pruned paths
        for x in self.parents.keys():
            _ = self[x]  # Evaluated for side-effect only

        yield from groups(self.parents).values()

    def union(self, *objects):
        """Find the sets containing the objects and merge them all."""
        # Find the heaviest root according to its weight.
        roots = iter(
            sorted(
                {self[x] for x in objects}, key=lambda r: self.weights[r], reverse=True
            )
        )
        try:
            root = next(roots)
        except StopIteration:
            return

        for r in roots:
            self.weights[root] += self.weights[r]
            self.parents[r] = root

In [382]:
x = UnionFind(maps.keys())

for i in bitmask:
    for j in maps.keys():
        if i^j in maps.keys():
            x.union(j,i^j)

In [383]:
k = []

for i in maps.keys():
    if x.__getitem__(i) not in k:
        k.append(x.__getitem__(i))
print(len(k))        

6118
